# 1. Import Distance Matrix and make it into a python list

In [11]:
#import dataset
import pandas as pd
import numpy as np

distance_matrix = pd.read_csv(f'D:\GitHub\waste_to_energy\d32f0742246d11ee\dataset\Distance_Matrix.csv', index_col=0)

# List of selected destination indices you want to display in the header
destination_indices = [317, 196, 9, 6, 36, 4, 3, 362, 10, 21, 281, 198, 319, 0, 14]

# Filter the distance matrix to include only the selected destinations in the header
selected_destinations = distance_matrix.iloc[destination_indices,:]
selected_destinations = selected_destinations.transpose()

# Remove the index and column names to get the raw numeric values as a numpy array
distances = np.round(selected_destinations.values,4)

# Convert the numpy array to a list of lists (list matrix)
costs = distances.tolist()

# 2. Import Biomass and make it a dictionary

In [12]:
#2018 is the new forecasted biomass
#import dataset
import pandas as pd
biomass_df = pd.read_csv("D:/GitHub/waste_to_energy/d32f0742246d11ee/dataset/forecast_result1.csv", index_col=0)

biomass_df.reset_index(inplace=True)
biomass_df['2018'] = biomass_df['2018'].round(6)

# Convert the 'biomass_df' DataFrame to a dictionary using 'Index' as keys and '2018' as values
biomass_supply = dict(zip(biomass_df['Index'].astype(str), biomass_df['2018']))

In [13]:
#Define Harvest Sites, Supply and Depots

harvest_sites = [str(i) for i in range(2418)]
supply = biomass_supply
depots = [str(value) for value in destination_indices]

# 3. LPP

In [14]:
# Import PuLP modeller functions
from pulp import *

demand_value = 20000

# Create a dictionary for the number of units of demand for each demand node
demand = {d: demand_value for d in depots}

# Find the length of the 'depots' list
length_of_depots = len(depots)


# Creates the prob variable to contain the problem data
prob = LpProblem("Biomass to Depot Transportation Problem", LpMinimize)


# Creates a list of tuples containing all the possible routes for transport
Routes = [(h, d) for h in harvest_sites for d in depots]

# A dictionary called route_vars is created to contain the referenced variables (the routes)
route_vars = LpVariable.dicts("Route", (harvest_sites, depots), 0, None, LpContinuous)

#LpVariable.dicts: This is a method provided by the PuLP library to create a dictionary of decision variables. 
# The variables are stored as a dictionary for easy referencing and access.

# Create a mapping from string keys to integer indices for costs
harvest_sites_indices = {h: i for i, h in enumerate(harvest_sites)}
depots_indices = {d: i for i, d in enumerate(depots)}

#1.
# The supply maximum constraints are added to prob for each supply node (harvest site)
# From each harvest site, the sum of biomass out of the harvest site must be less than or equal to the supply maximum for that harvest site
for h in harvest_sites:
    prob += lpSum([route_vars[h][d] for d in depots]) <= supply[h], f"Sum_of_biomass_out_of_harvestsite_{h}"

for h in harvest_sites:
    prob += lpSum([route_vars[h][d] for d in depots]) <= 20000, f"2Sum_of_biomass_out_of_harvestsite_{h}"

#2. 
# Take at least 80% of the total supply when we take the total sum at the demand end
total_sum = lpSum([route_vars[h][d] for h in harvest_sites for d in depots])
prob += total_sum >= 0.8 * lpSum([supply[h] for h in harvest_sites]), "80_percent_of_total_supply"

#3. 
for d in depots:
    prob += lpSum([route_vars[h][d] for h in harvest_sites]) == demand[d], f"Sum_of_biomass_at_depot_{d}"

# Minimize the total transportation cost and maximize the total sum of transported biomass
prob += 0.001*(lpSum([route_vars[h][d] * costs[harvest_sites_indices[h]][depots_indices[d]] for h in harvest_sites for d in depots])) + \
((demand_value*length_of_depots)-total_sum), "Combined_Objective_Function" #5 is the numberof depots
prob.solve()

d:\Softwares\Python\Python39\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


1

# 4. Print the results

In [15]:
# Print the results
print("Biomass to Depot Transportation SOLUTION -- Non-zero shipments")
print("TotalCost = ", value(prob.objective))
for (h, d) in Routes:
    if route_vars[h][d].value() > 0:
        print(f"Ship {route_vars[h][d].value()} unit of biomass from Harvest Site {h} to Depot {d}")
print()
print()

#Total Biomass Shipped:
sum1 = 0
for (h, d) in Routes:
    if route_vars[h][d].value() > 0:
        sum1 = sum1 + route_vars[h][d].value()
print("Total Biomass Shipped = ", sum1)

#80% of Total Biomass Available at all the harvest sites:
total_biomass = lpSum(supply.values())
print("80% of Total Biomass Available at all the harvest sites = ", 0.8*total_biomass)

Biomass to Depot Transportation SOLUTION -- Non-zero shipments
TotalCost =  90916.50684675241
Ship 7.18981 unit of biomass from Harvest Site 0 to Depot 0
Ship 41.751512 unit of biomass from Harvest Site 1 to Depot 0
Ship 56.298204 unit of biomass from Harvest Site 2 to Depot 0
Ship 78.114656 unit of biomass from Harvest Site 3 to Depot 3
Ship 18.028989 unit of biomass from Harvest Site 4 to Depot 4
Ship 24.384128 unit of biomass from Harvest Site 5 to Depot 4
Ship 44.649603 unit of biomass from Harvest Site 6 to Depot 6
Ship 61.254266 unit of biomass from Harvest Site 7 to Depot 6
Ship 34.910291 unit of biomass from Harvest Site 8 to Depot 6
Ship 45.991796 unit of biomass from Harvest Site 9 to Depot 9
Ship 11.684225 unit of biomass from Harvest Site 10 to Depot 10
Ship 6.516012 unit of biomass from Harvest Site 11 to Depot 10
Ship 6.390858 unit of biomass from Harvest Site 12 to Depot 10
Ship 0.156312 unit of biomass from Harvest Site 13 to Depot 10
Ship 0.547889 unit of biomass from 

In [ ]:
# Creating an empty list to store the results
results = []

# Iterating over the routes to extract non-zero shipments
for (h, d) in Routes:
    if route_vars[h][d].value() > 0:
        shipment_value = route_vars[h][d].value()
        results.append({
            "source_index": h,
            "destination_index": d,
            "value": shipment_value
        })

# Creating a DataFrame from the results list
results_df = pd.DataFrame(results)